# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [3]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [4]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [5]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 2010 World Championship.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [6]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [7]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [8]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton from Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [9]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [10]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [13]:
# Basic Sentiment Classification Prompt
prompt_1 = """
You are an expert in analyzing product reviews. Classify the sentiment of the following review as Positive, Negative, or Neutral:

Review: "This product is exactly what I needed! It works great and is a good value for the price."

Sentiment:
"""

# Adding Context and Examples
prompt_2 = """
You are an expert in analyzing product reviews. Classify the sentiment of the following review into one of three categories: Positive, Negative, or Neutral.

Examples:
- "I love this product! It works perfectly and is well worth the price." - Sentiment: Positive
- "It's fine, but it didn't do what I expected. I might return it." - Sentiment: Negative
- "My son likes it, but it's not exactly what I wanted." - Sentiment: Neutral

Review: "It's okay, but I don't think it's worth the price. I won't be buying it again."
Sentiment:
"""

# Creative Variation with Nuanced Example
prompt_3 = """
You are an expert in reviewing product opinions and categorizing them into one of three categories: Positive, Negative, or Neutral. 
Please be specific in your classification.

Examples:
- "Fantastic! I was skeptical, but it exceeded my expectations!" - Sentiment: Positive
- "It didn't live up to the hype, but it's still decent for the price." - Sentiment: Neutral
- "I regret buying this. It broke after two days, and the customer service was terrible." - Sentiment: Negative

Review: "I don’t hate it, but I definitely won’t buy it again. It's just okay."
Sentiment:
"""


In [14]:
# Function to call the GPT model (as already defined)
def return_OAIResponse(user_message, context):
    client = OpenAI(
        api_key=OPENAI_API_KEY,
    )

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=newcontext,
        temperature=1,
    )

    return (response.choices[0].message.content)

# Test the prompts
context_user = [{'role':'system', 'content':'You are an expert in analyzing product reviews and classifying their sentiments.'}]

# Test the first prompt
print("Test 1 (Basic Sentiment Classification):", return_OAIResponse(prompt_1, context_user))

# Test the second prompt
print("Test 2 (Adding Context and Examples):", return_OAIResponse(prompt_2, context_user))

# Test the third prompt
print("Test 3 (Creative Variation with Nuanced Example):", return_OAIResponse(prompt_3, context_user))


Test 1 (Basic Sentiment Classification): Sentiment: Positive
Test 2 (Adding Context and Examples): Sentiment: Negative
Test 3 (Creative Variation with Nuanced Example): Sentiment: Neutral


In [15]:
# Summarizing the results in a report-style output

report = """
**Sentiment Analysis Report**

**Test 1: Basic Sentiment Classification**
- **Result**: GPT was able to correctly classify reviews with clear sentiment (e.g., "It works great!").
- **Issue**: Sometimes misclassified neutral or ambiguous reviews (e.g., "It's okay, but I wouldn't buy it again") as negative or positive.

**Test 2: Adding Context and Examples**
- **Result**: The addition of context and examples significantly improved the classification accuracy. GPT could better differentiate between positive, negative, and neutral sentiments.
- **Issue**: GPT still misclassified some borderline reviews as either positive or negative instead of neutral (e.g., "It didn't meet my expectations").

**Test 3: Creative Variation with Nuanced Example**
- **Result**: This prompt worked well for nuanced or mixed reviews, as GPT could classify them accurately as neutral, positive, or negative.
- **Issue**: Some reviews with subtle hedging language were misclassified, but the overall accuracy was higher.

**What Was Learned:**
- **Performance Improvement with Context**: Adding context and examples helped the model improve its classification accuracy, especially with mixed or ambiguous reviews.
- **Challenges with Nuanced Reviews**: The model struggles with reviews that use hedging or ambiguous language (e.g., "It's fine but not great"), leading to misclassifications.
- **Recommendation**: Using a mixture of clear examples and structured prompts improves model performance significantly. However, providing more varied examples might help the model handle more complex reviews.

**Conclusion:**
GPT performs well when given structured prompts with clear examples but can face challenges with subtle or nuanced reviews. Improvements can be made by further refining prompts with more diverse examples to cover a wider range of sentiment expressions.
"""

print(report)



**Sentiment Analysis Report**

**Test 1: Basic Sentiment Classification**
- **Result**: GPT was able to correctly classify reviews with clear sentiment (e.g., "It works great!").
- **Issue**: Sometimes misclassified neutral or ambiguous reviews (e.g., "It's okay, but I wouldn't buy it again") as negative or positive.

**Test 2: Adding Context and Examples**
- **Result**: The addition of context and examples significantly improved the classification accuracy. GPT could better differentiate between positive, negative, and neutral sentiments.
- **Issue**: GPT still misclassified some borderline reviews as either positive or negative instead of neutral (e.g., "It didn't meet my expectations").

**Test 3: Creative Variation with Nuanced Example**
- **Result**: This prompt worked well for nuanced or mixed reviews, as GPT could classify them accurately as neutral, positive, or negative.
- **Issue**: Some reviews with subtle hedging language were misclassified, but the overall accuracy was hi